# LSTM parser for the French Treebank

This notebook trains a parser for the French Treebank. The parser is inspired by the one presented in Tsuruoka e.a. (2015), though it uses LSTM instead of conditional random fields. Like Tsuruoka e.a. (2015), the parser operates by a type of iterative chunking, continuing until it finds an analysis of the entire sentence.


##### Load library packages

In [25]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sys
import pickle


from keras.models import Model, load_model
from keras.layers import Bidirectional, Dense, Input, Dropout, LSTM, Activation, TimeDistributed, BatchNormalization, concatenate, Concatenate
from keras.layers.embeddings import Embedding
from keras.constraints import max_norm
from keras import regularizers
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.initializers import glorot_uniform
from keras import backend as K
from sklearn.model_selection import train_test_split
from gensim.models import KeyedVectors

from grail_data_utils import *

%matplotlib inline

np.random.seed(1)

### Read the parser actions file

The parser actions file contains triples of the form `Prosody|Formula|ParserAction`, where `Prosody` is either a word or a binary tree of words, `Formula` is formula from the TLGbank (simplified), and `ParserAction` gives the correct chunks for the current sequence, with `O` meaning a formula is outside of all chuncks.

In [52]:
def read_parsedata(file):
    with open(file, 'r') as f:
        vocabulary = set()
        vnorm = set()
        actionset = set()
        superset = set()
        sentno = 0
        maxlen = 0
        allpros = []
        allforms = []
        allactions = []
        for line in f:
            line = line.strip().split()
            length = len(line)
            sentforms = []
            sentpros = []
            sentactions = []
            if (length > maxlen):
                maxlen = length
            for l in range(length):
                item = line[l].split('|')
                pros = item[0]
                form = item[1]
                action = item[2]
                if not(pros.startswith("p(")):
                    nword = normalize_word(pros)
                    vocabulary.add(pros)
                    vnorm.add(nword)
                pros = pros.replace("p(0,", "")
                pros = pros.replace("p(1,", "")
                pros = pros.replace(")", "")
                pros = pros.split(",")
                superset.add(form)
                actionset.add(action)
                sentforms.append(form)
                sentpros.append(pros)
                sentactions.append(action)
            allpros.append(sentpros)
            allforms.append(sentforms)
            allactions.append(sentactions)
    X = np.asarray(allpros)
    Y = np.asarray(allforms)
    Z = np.asarray(allactions)
    return X, Y, Z, vocabulary, vnorm, superset, actionset, maxlen

In [53]:
# sentences with verified parses
# number of sentences, train: 9449, test: 3150, dev: 3150
pros, forms, actions, vocabulary, vnorm, superset, actionset, maxLen = read_parsedata('parse_all.txt')

In [15]:

print("Longest sentence   : ", maxLen)
print("Number of actions : ", len(actionset), actionset)
print("Number of supertags: ", len(superset), superset)


Longest sentence   :  140
Number of actions :  57 {'let1', 'dl2', 'wpop1', 'a_dit_se1', 'gap_i1', 'prod_i1', 'se_dit1', 'e_end1', 'prod_wl1', 'se_dit2', 'wpop_vpi1', 'dit_np2', 'O', 'gap_e1', 'prod_i2', 'wr_a2', 'dl1', 'c_r_lnr3', 'prod_dr1', 'prod_c1', 'e_end_r_lnr1', 'prod_dr2', 'gap_i2', 'e_endd1', 'e_endd2', 'a_dit2', 'a_dit1', 'c_r_lnr2', 'let2', 'c_l_lnr1', 'e_end_l_lnr2', 'e_end_l1', 'wpop_vpi2', 'e_start1', 'c_l_lnr3', 'prod_c2', 'c_r_lnr1', 'gap_c1', 'e_end_l2', 'e_end_r_lnr2', 'dit_np1', 'ef_start1', 'dr2', 'e_end_l_lnr1', 'prod_i3', 'e_end2', 'dr1', 'e_start_l1', 'wpop_vp1', 'wr1', 'prod_w1', 'ef_start_iv1', 'c_l_lnr2', 'wr2', 'wr_a1', 'a_dit_se2', 'prod_e1'}
Number of supertags:  509 {'dr(0,dl(0,dr(0,dr(0,dl(0,np,s),np),pp),dr(0,dr(0,dl(0,np,s),np),pp)),dr(0,dr(0,dl(0,np,s),np),pp))', 'dr(0,np,n)', 'dr(0,dl(0,pp,pp),s)', 'dl(0,np,dl(1,dl(0,n,n),dl(0,n,n)))', 'dl(0,n,pp)', 'dr(0,dl(0,s,s),dl(0,n,n))', 'p(0,dr(0,dl(0,np,s),np),dia(0,box(0,np)))', 'dr(0,dl(0,dr(0,pp,np),dl(0,n

## Create auxiliary mappings

Load vector embeddings for the vocabulary. Create mappings from words, formulas and actions integers and back.

In [26]:
wv = KeyedVectors.load_word2vec_format('../wang2vec/frwiki_cwindow50_10.bin', binary=True)
veclength = 50

In [29]:
def remove_prefix(text, prefix):
    if text.startswith(prefix):
        return text[len(prefix):]
    return text

Create vector mapping for all words in the vocabulary, using a zero vector for unknown words. Words are normalized before lookup (removing capitalization, etc.)

In [30]:
word_to_vec_map = {}
unknowns = set()
invoc = 0

for w in vocabulary:
    wn = normalize_word(w)
    wr = remove_prefix(wn, "-t-")
    wr = remove_prefix(wr, "-")
    try:
        vec = wv[wr]
        invoc = invoc + 1
    except:
        unknowns.add(w)
        vec = np.zeros(veclength)
    word_to_vec_map[w] = vec

print('Unknowns: ', len(unknowns))
print('In vocabulary: ', invoc)

Unknowns:  2720
In vocabulary:  11870


Compute a vector meaning for a complex prosodic term by taking the average for all words having an embedding. Other solutions, such as simple addition, are possible. An alternative would be to keep track only of the head word in each prosodic term (this will greatly reduce the embedding size and may work well)

In [43]:
def pros_to_vec(pros):
    vec = np.zeros(veclength)
    numw = 0
    for w in pros:
        wn = normalize_word(w)
        try:
            wvec = wv[wn]
            vec = vec + wvec
            numw = numw + 1
        except:
            pass
    if numw == 0:
        numw = 1
    return vec/numw    

Looks at some examples

In [23]:
print(pros[113])
print(len(pros[113]))
print(forms[113])
print(actions[113])
print(len(actions[113]))

[['Il'], ['y', 'a', 'quelques', 'jours'], ['', '', 'une', 'soixantaine', 'de', 'rebelles', 'présumés'], ['', '', 'dont', 'certains', 'étaient', 'armés'], ['', '', 'ont'], ['été', 'transférés', 'à', 'Abidjan'], ['.']]
7
['np', 'dl(0,np,dr(0,s,s))', 'np', 'dl(0,np,np)', 'dr(0,dl(0,np,s),dl(0,np,s))', 'dl(0,np,s)', 'dl(0,s,txt)']
['dl1', 'dl2', 'dl1', 'dl2', 'dr1', 'dr2', 'O']
7


In [44]:
print(pros_to_vec(pros[113][0]))
print(pros[113][0])
print(wv['il'])

[ 0.06135423 -0.13131003 -0.18451238 -0.25952235 -0.20443681  0.28091779
  0.12621945  0.15473232  0.09866685  0.02807902 -0.31461665 -0.08482077
 -0.05280184 -0.12457576 -0.39251816  0.03320421 -0.08924808 -0.59473646
 -0.16613822 -0.20295529  0.06712979  0.16169645 -0.13465753 -0.02375351
 -0.30984667  0.02895044 -0.03812085 -0.15557894 -0.42134857  0.2752746
 -0.63396919 -0.11106327  0.27074042  0.21478277 -0.09659664  0.44096583
 -0.02110181 -0.10309666  0.22942159 -0.05130152  0.44929028  0.04754487
  0.18998609 -0.02927026  0.43349707  0.20478503  0.25512281 -0.22025017
 -0.22031641  0.02538358]
['Il']
[ 0.06135423 -0.13131003 -0.18451238 -0.25952235 -0.20443681  0.28091779
  0.12621945  0.15473232  0.09866685  0.02807902 -0.31461665 -0.08482077
 -0.05280184 -0.12457576 -0.39251816  0.03320421 -0.08924808 -0.59473646
 -0.16613822 -0.20295529  0.06712979  0.16169645 -0.13465753 -0.02375351
 -0.30984667  0.02895044 -0.03812085 -0.15557894 -0.42134857  0.2752746
 -0.63396919 -0.1110

Create the mappings to and from integers

In [46]:
word_to_index, index_to_word = indexify(vocabulary)
form_to_index, index_to_form = indexify(superset)
action_to_index, index_to_action = indexify(actionset)

In [5]:
def load_obj(name):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)


## Create model inputs


In [56]:
numActions = len(index_to_action) + 1
numSuperClasses = len(index_to_form) + 1

Y = lists_to_indices(actions, action_to_index, maxLen)
X = lists_to_indices(forms, form_to_index, maxLen)

In [123]:
print(len(vocabulary))

14590


TypeError: object of type 'Word2VecKeyedVectors' has no len()

Keep 20% of the sentences as test data

In [57]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

Convert integer outputs to one-hot representations

In [117]:
Y_train_oh = to_categorical(Y_train, numActions)

## Define the model

As a baseline, use the simplest possible model with access only to the formulas but no information about the words.

In [58]:
sentence_indices = Input(shape = (maxLen,), dtype = 'int32')

In [59]:
emb = Embedding(numSuperClasses,64,trainable=True,mask_zero=True)(sentence_indices)

In [78]:
LSTM1 = Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))(emb)
LSTM2 = Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))(LSTM1)

In [111]:
Act =  TimeDistributed(Dense(numActions,kernel_constraint=max_norm(5.),activation='softmax'))(LSTM2)

Define the same model using a function 

In [112]:
def parse_model(input_shape, num_super, num_actions):
    sentence_indices = Input(shape = input_shape, dtype = 'int32')
    emb = Embedding(numSuperClasses,64,trainable=True,mask_zero=True)(sentence_indices)
    LSTM1 = Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))(emb)
    LSTM2 = Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))(LSTM1)
    Act =  TimeDistributed(Dense(numActions,kernel_constraint=max_norm(5.),activation='softmax'))(LSTM2)
    model = Model(inputs=sentence_indices,outputs=Act)
    
    return model

In [113]:
model = parse_model((maxLen,), numSuperClasses, numActions)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 140)               0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 140, 64)           32640     
_________________________________________________________________
bidirectional_9 (Bidirection (None, 140, 256)          197632    
_________________________________________________________________
bidirectional_10 (Bidirectio (None, 140, 256)          394240    
_________________________________________________________________
time_distributed_7 (TimeDist (None, 140, 58)           14906     
Total params: 639,418
Trainable params: 639,418
Non-trainable params: 0
_________________________________________________________________


In [120]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, Y_train_oh, epochs=10, batch_size=32,validation_split=0.25)

Train on 109235 samples, validate on 36412 samples
Epoch 1/10
109235/109235 [==============================] - 3204s 29ms/step - loss: 0.0628 - acc: 0.9784 - val_loss: 0.0439 - val_acc: 0.9843
Epoch 2/10
 34944/109235 [========>.....................] - ETA: 34:40 - loss: 0.0455 - acc: 0.9831

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model train vs validation loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model train vs validation accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
model.save('parse.h5')

In [116]:
print(model.layers[0].input_shape)
print(model.layers[0].output_shape)
print('Emb')
print(model.layers[1].input_shape)
print(model.layers[1].output_shape)
print('LSTM1')
print(model.layers[2].input_shape)
print(model.layers[2].output_shape)
print('LSTM2')
print(model.layers[3].input_shape)
print(model.layers[3].output_shape)
print('Act')
print(model.layers[4].input_shape)
print(model.layers[4].output_shape)
print('Numactions')
print(numActions)
print(Y_train.shape)

(None, 140)
(None, 140)
Emb
(None, 140)
(None, 140, 64)
LSTM1
(None, 140, 64)
(None, 140, 256)
LSTM2
(None, 140, 256)
(None, 140, 256)
Act
(None, 140, 256)
(None, 140, 58)
Numactions
58
(145647, 140)


In [71]:
print(X_train[0])
print(np.shape(X_train))
print(emb(X_train))

[   2.  422.  314.  422.  158.   31.  138.  422.    2.  422.  314.  422.
  314.  422.  278.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.]
(145647, 140)


TypeError: 'Tensor' object is not callable

In [128]:
Yin = Y[:2795]
print(np.shape(Yin))

(2795, 140)


In [26]:
def read_data(file):
    with open(file, 'r') as f:
        out = []
        for line in f:
            list = []
            line = line.strip().split()
            for i in line:
                list.append(i)
            out.append(list)
    return np.asarray(out) 

In [27]:
LeftList  = read_data('brackets_left.txt')
RightList = read_data('brackets_right.txt')

In [104]:
def l_to_indices(X, max_len):

    m = X.shape[0]                                   # number of training examples
    
    # Initialize X_indices as a numpy matrix of zeros and the correct shape (≈ 1 line)
    X_indices = np.zeros((m,max_len,1))

    for i in range(m):                               # loop over training examples
        
        # Convert the ith training sentence in lower case and split it into words. You should get a list of words.
        list = X[i]

        j = 0
        
        # Loop over the words of sentence_words
        for w in list:
            try:
                X_indices[i, j, 0] = float(w)
            except:
                print("Not a float/integer: ", w)
                X_indices[i, j, 0] = 0  # unknown
            # Increment j to j + 1
            j = j + 1
            
    return X_indices


In [105]:
Left = l_to_indices(LeftList, maxLen)
Right = l_to_indices(RightList, maxLen)

In [46]:
print(Left[0])
print(np.shape(Left))

[ 1.  2.  2.  1.  1.  1.  0.  1.  1.  1.  1.  1.  1.  0.  2.  0.  1.  2.
  1.  1.  1.  4.  0.  1.  2.  1.  1.  0.  0.  1.  2.  0.  1.  2.  1.  0.
  1.  1.  1.  0.  2.  0.  1.  3.  1.  0.  1.  0.  1.  2.  4.  0.  1.  0.
  1.  0.  1.  0.  2.  0.  1.  2.  0.  1.  0.  1.  2.  0.  2.  0.  2.  0.
  1.  1.  1.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
(2795, 140)


In [185]:
sentence_indices = Input(shape = (maxLen,), dtype = 'int32')

In [186]:
emb = Embedding(numSuperClasses,64,trainable=True,mask_zero=True)(sentence_indices)

In [187]:
X = Bidirectional(LSTM(256, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))(emb)
X = Bidirectional(LSTM(256, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))(X)

In [188]:
YL = TimeDistributed(Dense(32,kernel_constraint=max_norm(5.)))(X)
ZL = TimeDistributed(Dense(32,kernel_constraint=max_norm(5.)))(X)

In [189]:
L =  TimeDistributed(Dense(1,kernel_constraint=max_norm(5.)))(YL)
outl = Activation('relu')(L)

In [190]:
R =  TimeDistributed(Dense(1,kernel_constraint=max_norm(5.)))(ZL)
outr = Activation('relu')(R)

In [191]:
model = Model(sentence_indices, [outl, outr])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, 140)          0                                            
__________________________________________________________________________________________________
embedding_6 (Embedding)         (None, 140, 64)      57024       input_8[0][0]                    
__________________________________________________________________________________________________
bidirectional_14 (Bidirectional (None, 140, 512)     657408      embedding_6[0][0]                
__________________________________________________________________________________________________
bidirectional_15 (Bidirectional (None, 140, 512)     1574912     bidirectional_14[0][0]           
__________________________________________________________________________________________________
time_distr

In [192]:
model.compile(optimizer='rmsprop', loss=['mse','mse']) 

In [193]:
history = model.fit(Yin, [Left,Right], epochs=30, batch_size=32,validation_split=0.2)

Train on 2236 samples, validate on 559 samples
Epoch 1/30
2236/2236 [==============================] - 159s 71ms/step - loss: 6.9297 - activation_16_loss: 1.0293 - activation_17_loss: 5.9004 - val_loss: 5.0428 - val_activation_16_loss: 0.5925 - val_activation_17_loss: 4.4504
Epoch 2/30
2236/2236 [==============================] - 149s 67ms/step - loss: 2.8369 - activation_16_loss: 0.5996 - activation_17_loss: 2.2373 - val_loss: 2.1068 - val_activation_16_loss: 0.4636 - val_activation_17_loss: 1.6433
Epoch 3/30
2236/2236 [==============================] - 162s 72ms/step - loss: 1.9260 - activation_16_loss: 0.4413 - activation_17_loss: 1.4847 - val_loss: 1.7798 - val_activation_16_loss: 0.3378 - val_activation_17_loss: 1.4420
Epoch 4/30
2236/2236 [==============================] - 153s 69ms/step - loss: 1.6837 - activation_16_loss: 0.3338 - activation_17_loss: 1.3499 - val_loss: 2.2846 - val_activation_16_loss: 0.3057 - val_activation_17_loss: 1.9789
Epoch 5/30
2236/2236 [===============

In [197]:
Lpred, Rpred = model.predict(Y)

In [160]:
print(np.shape(Lpred))

(3320, 140, 1)


In [162]:
print(Lpred[1000,:,0])

[ 1.93247151  0.99550194  0.          3.08669186  0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.

In [201]:
def print_word(word):
    if word == "(":
        print('"("', end=' ')
    elif word == ")":
        print('")"', end=' ')
    else:
        print(word, end=' ')    
    

def print_prediction(ws, left, right):
    li = left.astype(int)
    ri = right.astype(int)
    for i in range(len(ws)):
        open = li[i]
        print("("*open, end=' ')
        print_word(ws[i])
        close = ri[i]
        print(")"*close, end=' ')

In [172]:
def parse_predict(sentno):
    print_prediction(words[sentno], Lpred[sentno,:,0], Rpred[sentno,:,0])
    print("")
    print_prediction(words[sentno], Left[sentno,:,0], Right[sentno,:,0])

In [203]:
parse_predict(2002)

( S'   il   confirme  ( son   objectif  ( de  ( freiner   la   progression   du   chômage ))))  en   fin   d'   année ))))))))) (( ,   il ) ( sait   aussi )  qu'  (( en  ( trois   mois ) ( 100.000  ( demandeurs   d'   emploi   supplémentaires )))  ont   été   recensés )))))))  .  
(( S'  ( il  ( confirme  ( son  ( objectif  ( de  (( freiner  ( la  ( progression  ( du   chômage )))) ( en  ( fin  ( d'   année )))))))))) (( ,   il ) (( sait   aussi ) ( qu'  (( en  ( trois   mois )) (( 100.000  (( demandeurs  ( d'   emploi ))  supplémentaires )) ( ont  ( été   recensés ))))))))  .  

In [168]:
print_prediction(words[1003], Lpred[1003,:,0], Rpred[1003,:,0])

(( Une  ( entreprise  ( de  ( conditionnement   , ))) (( France-Maîtrise   ,   serait  ( d'  (( accord   pour  ( démarrer   une )  entreprise  ( d'  ( insertion  ( sur   la )))))))  commune   .  

In [169]:
print_prediction(words[1003], Left[1003,:,0], Right[1003,:,0])

((( Une  ( entreprise  ( de   conditionnement ))) ( ,   France-Maîtrise )) ((( ,   serait ) ( d'   accord )) ( pour  (( démarrer  ( une  ( entreprise  ( d'   insertion )))) ( sur  ( la   commune ))))))  .  

In [148]:
print(words[1000])

['Son', 'rôle', ':', 'créer', 'une', 'synergie', 'entre', 'organismes', 'et', 'associations', '.']


In [131]:
print(Lpred[1000])

[[ 3.51808262]
 [ 0.        ]
 [ 1.06027424]
 [ 0.        ]
 [ 2.71995115]
 [ 0.        ]
 [ 2.04945779]
 [ 0.        ]
 [ 1.07729733]
 [ 1.14353406]
 [ 0.44215032]
 [ 1.10520411]
 [ 1.05459976]
 [ 0.        ]
 [ 2.13619494]
 [ 0.        ]
 [ 1.00193095]
 [ 0.99981958]
 [ 1.06076026]
 [ 1.04573667]
 [ 1.08062315]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.     

In [132]:
print(Rpred[0])

[[ 0.        ]
 [ 1.01456225]
 [ 0.        ]
 [ 2.68298173]
 [ 0.        ]
 [ 0.93558681]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 1.7421037 ]
 [ 0.        ]
 [ 0.        ]
 [ 6.3043251 ]
 [ 0.        ]
 [ 1.07220888]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 9.12233067]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.     